In [1]:
import os
import torch
import torchvision

from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

from model import PlaceholderModel
from pathlib import Path
from argparse import Namespace

/Users/nikhilvs/anaconda3/envs/dl/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


In [2]:
algo = "conv"
architecture = "arch_1"

data_path = '../../data/Proca_fiducial_scaled_cropped.hdf5'
# data_path = "/scratch/prs392/research_internship/data/sketch_recognition/tu_berlin/png_split/"
checkpoint_path = '/Users/nikhilvs/checkpoints'

experiment_name = 'tuning'

d = os.path.join(checkpoint_path, experiment_name)
Path(d).mkdir(parents=True, exist_ok=True)
versions = [o for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]

versions = sorted(versions)

In [3]:
checkpoint_path

'/Users/nikhilvs/checkpoints'

In [4]:
from tensorboard.backend.event_processing import event_accumulator
import numpy as np

def _load_run(path):
    event_acc = event_accumulator.EventAccumulator(path)
    event_acc.Reload()
    data = {}

    for tag in sorted(event_acc.Tags()["scalars"]):
        x, y = [], []

        for scalar_event in event_acc.Scalars(tag):
            x.append(scalar_event.step)
            y.append(scalar_event.value)

        data[tag] = (np.asarray(x), np.asarray(y))
    return data

In [5]:
from pytorch_lightning.core.saving import load_hparams_from_yaml

list_existing_hparams = []
list_of_val_loss = []

for version in versions:
    hparam_path = os.path.join(checkpoint_path, experiment_name, version, 'hparams.yaml')
    hparams_new = load_hparams_from_yaml(hparam_path)

    print(hparams_new)
    print(min(_load_run(os.path.join(checkpoint_path, experiment_name, version))['val_loss'][1]))

    list_existing_hparams.append(hparams_new)
    list_of_val_loss.append(min(_load_run(os.path.join(checkpoint_path, experiment_name, version))['val_loss'][1]))

{'batch_size': 16}


/Users/nikhilvs/anaconda3/envs/dl/lib/python3.7/site-packages/pytorch_lightning/core/saving.py:335: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  tags = yaml.load(fp)


KeyError: 'val_loss'

In [6]:
seed_everything(123)

parameterization = {
    'batch_size' : 16
}

print(parameterization)

hparams = Namespace(**parameterization)
model = PlaceholderModel(hparams, data_path)
logger = TensorBoardLogger(checkpoint_path, name=experiment_name)
checkpoint_callback = ModelCheckpoint(
    filepath=None,
    save_top_k=5,
    save_last = True,
    verbose=False,
    monitor='val_loss',
    mode='min',
    prefix=''
)
trainer = Trainer(
     logger = logger, 
     default_root_dir=checkpoint_path,
     checkpoint_callback = checkpoint_callback,
     max_epochs = 2,
#      gpus = -1,
#      distributed_backend = 'dp',
     val_check_interval=0.10,
     log_save_interval=1,
     row_log_interval=1,
     fast_dev_run=False
)
trainer.fit(model)

{'batch_size': 16}


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/Users/nikhilvs/anaconda3/envs/dl/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name      | Type       | Params
-----------------------------------------
0 | encoder   | Sequential | 765 M 
1 | decoder   | Sequential | 765 M 
2 | criterion | MSELoss    | 0     


/Users/nikhilvs/anaconda3/envs/dl/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Saving latest checkpoint..
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/nikhilvs/anaconda3/envs/dl/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/Users/nikhilvs/anaconda3/envs/dl/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/Users/nikhilvs/anaconda3/envs/dl/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/nikhilvs/anaconda3/envs/dl/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/nikhilvs/anaconda3/envs/dl/lib/python3.7/multiprocessing/connection.py", line 368, in _send


1